In [ ]:
import os
import sys
import matplotlib
matplotlib.rcParams['savefig.dpi'] = 100

from tempfile import NamedTemporaryFile
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import display, HTML
import numpy as np
import base64

IMG_TAG = """<img src="data:image/gif;base64,{0}" alt="some_text">"""
 
def anim_to_gif(anim):
    data="0"
    with NamedTemporaryFile(suffix='.gif') as f:
        anim.save(f.name, writer='imagemagick', fps=10);
        print(f.name)
        data = open(f.name, "rb").read()
        #print(data)
        data = base64.b64encode(data).decode()
        #print(data)
    return IMG_TAG.format(data)
 
def display_animation(anim):
    plt.close(anim._fig)
    return display(HTML(anim_to_gif(anim)))

In [ ]:
dirname='/d/d9/ychen/2018_production_runs/20180802_L438_rc10_beta07/'
simtitle = dirname.split('/')[-1]
nozzledata = np.genfromtxt(os.path.join(dirname, 'nozzleVec.dat'))
print(nozzledata.shape)
nstep = nozzledata[:,0]
ttnoz = nozzledata[:,1]
xx = nozzledata[:,2]
yy = nozzledata[:,3]
zz = nozzledata[:,4]
thetas = np.arccos(zz)/np.pi*180
phis = np.arctan2(yy, xx)

In [ ]:
endstep = 150000
dstep = 1000

# Create new Figure and an Axes which fills it.
fig = plt.figure(figsize=(6,6))
ax = plt.subplot(111, polar=True)

line, = ax.plot(phis[:endstep], thetas[:endstep], alpha=0.6)
ax.set_title(simtitle,loc='left')
ax.set_rlim(0,12)




def update(step):
    sys.stdout.write('\r step: %6i / %6i' % (step, endstep))
    line.set_xdata(phis[:step])
    line.set_ydata(thetas[:step])
    time_prv = ttnoz[step-dstep]/3.15569E13
    time = ttnoz[step]/3.15569E13
    tmyr = int(time)
    if time_prv < tmyr and time > tmyr:
        arg = np.argmin(abs(ttnoz-tmyr*3.15569E13))
        ax.scatter(phis[arg], thetas[arg], marker='x', color='r', s=4)
        ax.annotate('%i Myr' % tmyr, xy=(phis[arg], thetas[arg]), size=6, bbox=dict(facecolor='w', lw=0, alpha=0.6))
   
    
ani = animation.FuncAnimation(fig, update, frames=range(0, endstep, dstep), interval=10)

display_animation(ani)